In [1]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition
solver = SolverFactory("cplex")

In [2]:
windgeneration = pd.read_csv('Average Capacities Summer Winter Wind.csv')
windgeneration.columns[1:]
windgen = windgeneration.to_dict()

solargeneration = pd.read_csv('Average Capacities Summer Winter Solar.csv')
solargeneration.columns[1:]
solargen = solargeneration.to_dict()

In [3]:
#df = pd.read_csv('pyomo optimisation - summer and winter in a day - low case.csv')
#df = pd.read_csv('pyomo optimisation - summer and winter in a day - median case.csv')
df = pd.read_csv('pyomo optimisation - summer and winter in a day - high case.csv')
dict_demand = dict(zip(df['Hour'], df['Demand (kW)']))

solar_lcoe = 45/1000
wind_lcoe = 85/1000
battery_lcoe = 120/1000 
ng_lcoe = 124/1000

wind_capacity_var_1277_lcoe       = 75.416/ 1000
wind_capacity_var_154_lcoe        = 74.17 / 1000
wind_capacity_var_1700_lcoe       = 105.26/ 1000
wind_capacity_var_1737_lcoe       = 103.92/ 1000
wind_capacity_var_1804point5_lcoe = 73.70 / 1000
wind_capacity_var_1804_lcoe       = 93.24 / 1000
wind_capacity_var_2048_lcoe       = 80.67 / 1000
wind_capacity_var_209_lcoe        = 75.41 / 1000
wind_capacity_var_249_lcoe        = 102.2 / 1000
wind_capacity_var_263_lcoe        = 73.60 / 1000
wind_capacity_var_356_lcoe        = 80.64 / 1000
wind_capacity_var_523_lcoe        = 74.52 / 1000
wind_capacity_var_530_lcoe        = 73.12 / 1000
wind_capacity_var_622_lcoe        = 90.95 / 1000
wind_capacity_var_739_lcoe        = 85.5  / 1000

solar_capacity_var_92_lcoe         = 48.27591831 / 1000
solar_capacity_var_170_lcoe        = 48.60209126 / 1000
solar_capacity_var_110_lcoe        = 47.91957056 / 1000
solar_capacity_var_94_lcoe         = 48.26147042 / 1000
solar_capacity_var_214_lcoe        = 48.41273781 / 1000
solar_capacity_var_38_lcoe         = 47.98403999 / 1000
solar_capacity_var_21_lcoe         = 47.97827406 / 1000
solar_capacity_var_112_lcoe        = 48.21579899 / 1000
solar_capacity_var_107_lcoe        = 47.99562687 / 1000
solar_capacity_var_213_lcoe        = 47.82156755 / 1000

model = ConcreteModel(name="Optimising Costs")

timesteps = 48 #hours
model.time_set = Set(initialize=range(0, timesteps))

model.time_set_winter = Set(initialize=range(0, 24), within= model.time_set)
model.time_set_summer = Set(initialize=range(24, timesteps), within = model.time_set)


#LiIon variables
charging_eff = 0.95
discharging_eff = 1.05
self_discharge = 0.002

overgeneration = 1.05#1.05
initial_soc = 0

#natgas_capacity_var = 3150000
natgas_capacity_var = 5430000

In [4]:
#model.solar_capacity_var = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var = pyo.Var(within=NonNegativeReals,initialize = 100)

model.batt_kWh_var = pyo.Var(within=NonNegativeReals,initialize = 100)
model.batt_kW_var = pyo.Var(within=NonNegativeReals,initialize = 100)


In [5]:
#model.solar_gen_var = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.ng_gen_var = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)

model.batt_soc_var = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.batt_charging_rate = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.batt_discharging_rate = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)

model.solar_gen_var_92        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_177       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_110       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_94        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_214       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_38        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_21        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_112       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_107       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.solar_gen_var_213       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)


model.wind_gen_var_1277       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_154        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_1700       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_1737       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_1804point5 = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_1804       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_2048       = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_209        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_249        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_263        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_356        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_523        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_530        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_622        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)
model.wind_gen_var_739        = pyo.Var(model.time_set,within=NonNegativeReals,initialize = 0)


model.solar_capacity_var_92  = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_177 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_110 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_94  = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_214 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_38  = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_21  = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_112 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_107 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.solar_capacity_var_213 = pyo.Var(within=NonNegativeReals,initialize = 100)

model.wind_capacity_var_1277       = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_154        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_1700       = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_1737       = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_1804point5 = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_1804       = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_2048       = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_209        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_249        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_263        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_356        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_523        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_530        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_622        = pyo.Var(within=NonNegativeReals,initialize = 100)
model.wind_capacity_var_739        = pyo.Var(within=NonNegativeReals,initialize = 100)

In [6]:
def s_gen_rule_92_win(self,t):
    return model.solar_gen_var_92[t] == model.solar_capacity_var_92 * solargen['92 Jan CF'][t]
model.s_gen_rule_92_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_92_win)

def s_gen_rule_92_summer(self,t):
    return model.solar_gen_var_92[t] == model.solar_capacity_var_92 * solargen['92 August CF'][t - 24]
model.s_gen_rule_92_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_92_summer)

def s_gen_rule_177_win(self,t):
    return model.solar_gen_var_177[t] == model.solar_capacity_var_177 * solargen['177 Jan CF'][t]
model.s_gen_rule_177_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_177_win)

def s_gen_rule_177_summer(self,t):
    return model.solar_gen_var_177[t] == model.solar_capacity_var_177 * solargen['177 August CF'][t - 24]
model.s_gen_rule_177_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_177_summer)

def s_gen_rule_110_win(self,t):
    return model.solar_gen_var_110[t] == model.solar_capacity_var_110 * solargen['110 Jan CF'][t]
model.s_gen_rule_110_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_110_win)

def s_gen_rule_110_summer(self,t):
    return model.solar_gen_var_110[t] == model.solar_capacity_var_110 * solargen['110 August CF'][t - 24]
model.s_gen_rule_110_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_110_summer)

def s_gen_rule_94_win(self,t):
    return model.solar_gen_var_94[t] == model.solar_capacity_var_94 * solargen['94 Jan CF'][t]
model.s_gen_rule_94_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_94_win)

def s_gen_rule_94_summer(self,t):
    return model.solar_gen_var_94[t] == model.solar_capacity_var_94 * solargen['94 August CF'][t - 24]
model.s_gen_rule_94_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_94_summer)

def s_gen_rule_214_win(self,t):
    return model.solar_gen_var_214[t] == model.solar_capacity_var_214 * solargen['214 Jan CF'][t]
model.s_gen_rule_214_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_214_win)

def s_gen_rule_214_summer(self,t):
    return model.solar_gen_var_214[t] == model.solar_capacity_var_214 * solargen['214 August CF'][t - 24]
model.s_gen_rule_214_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_214_summer)

def s_gen_rule_38_win(self,t):
    return model.solar_gen_var_38[t] == model.solar_capacity_var_38 * solargen['38 Jan CF'][t]
model.s_gen_rule_38_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_38_win)

def s_gen_rule_38_summer(self,t):
    return model.solar_gen_var_38[t] == model.solar_capacity_var_38 * solargen['38 August CF'][t - 24]
model.s_gen_rule_38_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_38_summer)

def s_gen_rule_21_win(self,t):
    return model.solar_gen_var_21[t] == model.solar_capacity_var_21 * solargen['21 Jan CF'][t]
model.s_gen_rule_21_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_21_win)

def s_gen_rule_21_summer(self,t):
    return model.solar_gen_var_21[t] == model.solar_capacity_var_21 * solargen['21 August CF'][t - 24]
model.s_gen_rule_21_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_21_summer)

def s_gen_rule_112_win(self,t):
    return model.solar_gen_var_112[t] == model.solar_capacity_var_112 * solargen['112 Jan CF'][t]
model.s_gen_rule_112_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_112_win)

def s_gen_rule_112_summer(self,t):
    return model.solar_gen_var_112[t] == model.solar_capacity_var_112 * solargen['112 August CF'][t - 24]
model.s_gen_rule_112_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_112_summer)

def s_gen_rule_107_win(self,t):
    return model.solar_gen_var_107[t] == model.solar_capacity_var_107 * solargen['107 Jan CF'][t]
model.s_gen_rule_107_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_107_win)

def s_gen_rule_107_summer(self,t):
    return model.solar_gen_var_107[t] == model.solar_capacity_var_107 * solargen['107 August CF'][t - 24]
model.s_gen_rule_107_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_107_summer)

def s_gen_rule_213_win(self,t):
    return model.solar_gen_var_213[t] == model.solar_capacity_var_213 * solargen['213 Jan CF'][t]
model.s_gen_rule_213_win_eql = pyo.Constraint(model.time_set_winter, rule=s_gen_rule_213_win)

def s_gen_rule_213_summer(self,t):
    return model.solar_gen_var_213[t] == model.solar_capacity_var_213 * solargen['213 August CF'][t - 24]
model.s_gen_rule_213_summer_eql = pyo.Constraint(model.time_set_summer, rule=s_gen_rule_213_summer)


In [7]:
def w_gen_rule_1277_win(self,t):
    return model.wind_gen_var_1277[t] == model.wind_capacity_var_1277 * windgen['1277 Jan CF'][t]
model.w_gen_rule_1277_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_1277_win)

def w_gen_rule_1277_summer(self,t):
    return model.wind_gen_var_1277[t] == model.wind_capacity_var_1277 * windgen['1277 August CF'][t - 24]
model.w_gen_rule_1277_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_1277_summer)

def w_gen_rule_154_win(self,t):
    return model.wind_gen_var_154[t] == model.wind_capacity_var_154 * windgen['154 Jan CF'][t]
model.w_gen_rule_154_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_154_win)

def w_gen_rule_154_summer(self,t):
    return model.wind_gen_var_154[t] == model.wind_capacity_var_154 * windgen['154 August CF'][t - 24]
model.w_gen_rule_154_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_154_summer)

def w_gen_rule_1700_win(self,t):
    return model.wind_gen_var_1700[t] == model.wind_capacity_var_1700 * windgen['1700 Jan CF'][t]
model.w_gen_rule_1700_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_1700_win)

def w_gen_rule_1700_summer(self,t):
    return model.wind_gen_var_1700[t] == model.wind_capacity_var_1700 * windgen['1700 August CF'][t - 24]
model.w_gen_rule_1700_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_1700_summer)

def w_gen_rule_1737_win(self,t):
    return model.wind_gen_var_1737[t] == model.wind_capacity_var_1737 * windgen['1737 Jan CF'][t]
model.w_gen_rule_1737_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_1737_win)

def w_gen_rule_1737_summer(self,t):
    return model.wind_gen_var_1737[t] == model.wind_capacity_var_1737 * windgen['1737 August CF'][t - 24]
model.w_gen_rule_1737_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_1737_summer)

def w_gen_rule_1804point5_win(self,t):
    return model.wind_gen_var_1804point5[t] == model.wind_capacity_var_1804point5 * windgen['1804.5 Jan CF'][t]
model.w_gen_rule_1804point5_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_1804point5_win)

def w_gen_rule_1804point5_summer(self,t):
    return model.wind_gen_var_1804point5[t] == model.wind_capacity_var_1804point5 * windgen['1804.5 August CF'][t - 24]
model.w_gen_rule_1804point5_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_1804point5_summer)

def w_gen_rule_1804_win(self,t):
    return model.wind_gen_var_1804[t] == model.wind_capacity_var_1804 * windgen['1804 Jan CF'][t]
model.w_gen_rule_1804_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_1804_win)

def w_gen_rule_1804_summer(self,t):
    return model.wind_gen_var_1804[t] == model.wind_capacity_var_1804 * windgen['1804 August CF'][t - 24]
model.w_gen_rule_1804_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_1804_summer)

def w_gen_rule_2048_win(self,t):
    return model.wind_gen_var_2048[t] == model.wind_capacity_var_2048 * windgen['2048 Jan CF'][t]
model.w_gen_rule_2048_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_2048_win)

def w_gen_rule_2048_summer(self,t):
    return model.wind_gen_var_2048[t] == model.wind_capacity_var_2048 * windgen['2048 August CF'][t - 24]
model.w_gen_rule_2048_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_2048_summer)

def w_gen_rule_209_win(self,t):
    return model.wind_gen_var_209[t] == model.wind_capacity_var_209 * windgen['209 Jan CF'][t]
model.w_gen_rule_209_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_209_win)

def w_gen_rule_209_summer(self,t):
    return model.wind_gen_var_209[t] == model.wind_capacity_var_209 * windgen['209 August CF'][t - 24]
model.w_gen_rule_209_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_209_summer)

def w_gen_rule_249_win(self,t):
    return model.wind_gen_var_249[t] == model.wind_capacity_var_249 * windgen['249 Jan CF'][t]
model.w_gen_rule_249_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_249_win)

def w_gen_rule_249_summer(self,t):
    return model.wind_gen_var_249[t] == model.wind_capacity_var_249 * windgen['249 August CF'][t - 24]
model.w_gen_rule_249_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_249_summer)

def w_gen_rule_263_win(self,t):
    return model.wind_gen_var_263[t] == model.wind_capacity_var_263 * windgen['263 Jan CF'][t]
model.w_gen_rule_263_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_263_win)

def w_gen_rule_263_summer(self,t):
    return model.wind_gen_var_263[t] == model.wind_capacity_var_263 * windgen['263 August CF'][t - 24]
model.w_gen_rule_263_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_263_summer)

def w_gen_rule_356_win(self,t):
    return model.wind_gen_var_356[t] == model.wind_capacity_var_356 * windgen['356 Jan CF'][t]
model.w_gen_rule_356_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_356_win)

def w_gen_rule_356_summer(self,t):
    return model.wind_gen_var_356[t] == model.wind_capacity_var_356 * windgen['356 August CF'][t - 24]
model.w_gen_rule_356_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_356_summer)

def w_gen_rule_523_win(self,t):
    return model.wind_gen_var_523[t] == model.wind_capacity_var_523 * windgen['523 Jan CF'][t]
model.w_gen_rule_523_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_523_win)

def w_gen_rule_523_summer(self,t):
    return model.wind_gen_var_523[t] == model.wind_capacity_var_523 * windgen['523 August CF'][t - 24]
model.w_gen_rule_523_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_523_summer)

def w_gen_rule_530_win(self,t):
    return model.wind_gen_var_530[t] == model.wind_capacity_var_530 * windgen['530 Jan CF'][t]
model.w_gen_rule_530_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_530_win)

def w_gen_rule_530_summer(self,t):
    return model.wind_gen_var_530[t] == model.wind_capacity_var_530 * windgen['530 August CF'][t - 24]
model.w_gen_rule_530_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_530_summer)

def w_gen_rule_622_win(self,t):
    return model.wind_gen_var_622[t] == model.wind_capacity_var_622 * windgen['622 Jan CF'][t]
model.w_gen_rule_622_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_622_win)

def w_gen_rule_622_summer(self,t):
    return model.wind_gen_var_622[t] == model.wind_capacity_var_622 * windgen['622 August CF'][t - 24]
model.w_gen_rule_622_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_622_summer)

def w_gen_rule_739_win(self,t):
    return model.wind_gen_var_739[t] == model.wind_capacity_var_739 * windgen['739 Jan CF'][t]
model.w_gen_rule_739_win_eql = pyo.Constraint(model.time_set_winter, rule=w_gen_rule_739_win)

def w_gen_rule_739_summer(self,t):
    return model.wind_gen_var_739[t] == model.wind_capacity_var_739 * windgen['739 August CF'][t - 24]
model.w_gen_rule_739_summer_eql = pyo.Constraint(model.time_set_summer, rule=w_gen_rule_739_summer)


In [8]:
def wind_capacity_var_1277_rule(self,t):
    return model.wind_capacity_var_1277 <= 1277*1000
model.wind_capacity_var_1277_rule_eql = pyo.Constraint(rule=wind_capacity_var_1277_rule)

def wind_capacity_var_154_rule(self,t):
    return model.wind_capacity_var_154 <= 154*1000
model.wind_capacity_var_154_rule_eql = pyo.Constraint(rule=wind_capacity_var_154_rule)

def wind_capacity_var_1700_rule(self,t):
    return model.wind_capacity_var_1700 <= 1700*1000
model.wind_capacity_var_1700_rule_eql = pyo.Constraint(rule=wind_capacity_var_1700_rule)

def wind_capacity_var_1737_rule(self,t):
    return model.wind_capacity_var_1737 <= 1737*1000
model.wind_capacity_var_1737_rule_eql = pyo.Constraint(rule=wind_capacity_var_1737_rule)

def wind_capacity_var_1804point5_rule(self,t):
    return model.wind_capacity_var_1804point5 <= 1804.5*1000
model.wind_capacity_var_1804point5_rule_eql = pyo.Constraint(rule=wind_capacity_var_1804point5_rule)

def wind_capacity_var_1804_rule(self,t):
    return model.wind_capacity_var_1804 <= 1804*1000
model.wind_capacity_var_1804_rule_eql = pyo.Constraint(rule=wind_capacity_var_1804_rule)

def wind_capacity_var_2048_rule(self,t):
    return model.wind_capacity_var_2048 <= 2048*1000
model.wind_capacity_var_2048_rule_eql = pyo.Constraint(rule=wind_capacity_var_2048_rule)

def wind_capacity_var_209_rule(self,t):
    return model.wind_capacity_var_209 <= 209*1000
model.wind_capacity_var_209_rule_eql = pyo.Constraint(rule=wind_capacity_var_209_rule)

def wind_capacity_var_249_rule(self,t):
    return model.wind_capacity_var_249 <= 249*1000
model.wind_capacity_var_249_rule_eql = pyo.Constraint(rule=wind_capacity_var_249_rule)

def wind_capacity_var_263_rule(self,t):
    return model.wind_capacity_var_263 <= 263*1000
model.wind_capacity_var_263_rule_eql = pyo.Constraint(rule=wind_capacity_var_263_rule)

def wind_capacity_var_356_rule(self,t):
    return model.wind_capacity_var_356 <= 356*1000
model.wind_capacity_var_356_rule_eql = pyo.Constraint(rule=wind_capacity_var_356_rule)

def wind_capacity_var_523_rule(self,t):
    return model.wind_capacity_var_523 <= 523*1000
model.wind_capacity_var_523_rule_eql = pyo.Constraint(rule=wind_capacity_var_523_rule)

def wind_capacity_var_530_rule(self,t):
    return model.wind_capacity_var_530 <= 530*1000
model.wind_capacity_var_530_rule_eql = pyo.Constraint(rule=wind_capacity_var_530_rule)

def wind_capacity_var_622_rule(self,t):
    return model.wind_capacity_var_622 <= 622*1000
model.wind_capacity_var_622_rule_eql = pyo.Constraint(rule=wind_capacity_var_622_rule)

def wind_capacity_var_739_rule(self,t):
    return model.wind_capacity_var_739 <= 739*1000
model.wind_capacity_var_739_rule_eql = pyo.Constraint(rule=wind_capacity_var_739_rule)

In [9]:
def ng_gen_rule(self,t):
    return model.ng_gen_var[t] <= natgas_capacity_var
model.ng_gen_eql = pyo.Constraint(model.time_set, rule=ng_gen_rule)

In [10]:
def solar_capacity_var_92_rule(self,t):
    return model.solar_capacity_var_92  <= 92 * 1000
model.solar_capacity_var_92_rule_eql = pyo.Constraint(rule =  solar_capacity_var_92_rule)

def solar_capacity_var_177_rule(self,t):
    return model.solar_capacity_var_177 <= 177 * 1000
model.solar_capacity_var_177_rule_eql = pyo.Constraint(rule = solar_capacity_var_177_rule )

def solar_capacity_var_110_rule(self,t):
    return model.solar_capacity_var_110 <= 110 * 1000
model.solar_capacity_var_110_rule_eql = pyo.Constraint(rule = solar_capacity_var_110_rule )

def solar_capacity_var_94_rule(self,t):
    return model.solar_capacity_var_94 <= 94 * 1000
model.solar_capacity_var_94_rule_eql = pyo.Constraint(rule =  solar_capacity_var_94_rule)

def solar_capacity_var_214_rule(self,t):
    return model.solar_capacity_var_214 <= 214 * 1000
model.solar_capacity_var_214_rule_eql = pyo.Constraint(rule = solar_capacity_var_214_rule )

def solar_capacity_var_38_rule(self,t):
    return model.solar_capacity_var_38 <= 38 * 1000
model.solar_capacity_var_38_rule_eql = pyo.Constraint(rule =  solar_capacity_var_38_rule)

def solar_capacity_var_21_rule(self,t):
    return model.solar_capacity_var_21 <= 21 * 1000
model.solar_capacity_var_21_rule_eql = pyo.Constraint(rule =  solar_capacity_var_21_rule)

def solar_capacity_var_112_rule(self,t):
    return model.solar_capacity_var_112 <= 112 * 1000
model.solar_capacity_var_112_rule_eql = pyo.Constraint(rule = solar_capacity_var_112_rule )

def solar_capacity_var_107_rule(self,t):
    return model.solar_capacity_var_107 <= 107 * 1000
model.solar_capacity_var_107_rule_eql = pyo.Constraint(rule = solar_capacity_var_107_rule )

def solar_capacity_var_213_rule(self,t):
    return model.solar_capacity_var_213 <= 213 * 1000
model.solar_capacity_var_213_rule_eql = pyo.Constraint(rule = solar_capacity_var_213_rule )

In [11]:
def batt_c_rate_rule(self,t):
    return model.batt_kW_var == model.batt_kWh_var / 2.0
model.batt_c_rate_eql =  pyo.Constraint(rule=batt_c_rate_rule)
#Based on energy-to-power ratio of 2 

In [12]:
def batt_soc_changes(self,t):
    if(t == 0):
        return model.batt_soc_var[t] == initial_soc*model.batt_kWh_var
    return model.batt_soc_var[t] == model.batt_soc_var[t-1]*(1-self_discharge) + (model.batt_charging_rate[t]*charging_eff - model.batt_discharging_rate[t])
model.batt_soc_changes_eql = pyo.Constraint(model.time_set, rule=batt_soc_changes)

In [13]:
def batt_soc_end_day1(self,t):
    return model.batt_soc_var[int(0.5*timesteps - 1)] == initial_soc*model.batt_kWh_var
model.batt_soc_end_day1_eql = pyo.Constraint(model.time_set, rule=batt_soc_end_day1)

def batt_soc_end_day2(self,t):
    return model.batt_soc_var[timesteps - 1] == initial_soc*model.batt_kWh_var
model.batt_soc_end_day2_eql = pyo.Constraint(model.time_set, rule=batt_soc_end_day2)

In [14]:
def batt_soc_max_kWh(self,t):
    return model.batt_soc_var[t] <= model.batt_kWh_var
model.batt_soc_max_kWh_eql = pyo.Constraint(model.time_set, rule=batt_soc_max_kWh)

In [15]:
def max_charging_rate(self,t):
    return model.batt_charging_rate[t] <= model.batt_kW_var
model.max_charging_rate_eql = pyo.Constraint(model.time_set, rule=max_charging_rate)

def max_discharging_rate(self,t):
    return model.batt_discharging_rate[t] <= model.batt_kW_var
model.max_discharging_rate_eql = pyo.Constraint(model.time_set, rule=max_discharging_rate)

In [16]:
def battery_charging_rule (self,t):
    if(t == 0):
        return model.batt_charging_rate[t] <= 0.5*(model.batt_kWh_var)
    return model.batt_charging_rate[t] <= (model.batt_kWh_var - model.batt_soc_var[t - 1])
model.battery_charging_eql = pyo.Constraint(model.time_set, rule=battery_charging_rule)

def battery_discharging_rule (self,t):
    if(t == 0):
        return model.batt_discharging_rate[t] <= 0.5*(model.batt_kWh_var)
    return model.batt_discharging_rate[t] <= model.batt_soc_var[t - 1]
model.battery_discharging_eql = pyo.Constraint(model.time_set, rule=battery_discharging_rule)

In [17]:
def demand_must_meet_day1 (self,t):
    return sum(model.solar_gen_var_92[t] + model.solar_gen_var_177[t] + model.solar_gen_var_110[t] \
               + model.solar_gen_var_94[t] + model.solar_gen_var_214[t] + model.solar_gen_var_38[t] \
               + model.solar_gen_var_21[t] + model.solar_gen_var_112[t] + model.solar_gen_var_107[t] \
               + model.solar_gen_var_107[t] + model.solar_gen_var_213[t] + model.wind_gen_var_1277[t] \
               + model.wind_gen_var_154[t] + model.wind_gen_var_1700[t] + model.wind_gen_var_1737[t] \
               + model.wind_gen_var_1804point5[t] + model.wind_gen_var_1804[t] + model.wind_gen_var_2048[t] \
               + model.wind_gen_var_209[t] + model.wind_gen_var_249[t] + model.wind_gen_var_263[t] \
               + model.wind_gen_var_356[t] + model.wind_gen_var_523[t] + model.wind_gen_var_530[t] \
               + model.wind_gen_var_622[t] + model.wind_gen_var_739[t] + model.ng_gen_var[t] for t in model.time_set_winter) \
               + sum(model.batt_soc_var[t] - model.batt_soc_var[t-1] for t in range(1,24)) \
                     >= sum(dict_demand[t] for t in model.time_set_winter)

model.demand_condition_day1_eql = pyo.Constraint(rule=demand_must_meet_day1)

def demand_must_meet_day2 (self,t):
    return sum(model.solar_gen_var_92[t] + model.solar_gen_var_177[t] + model.solar_gen_var_110[t] \
               + model.solar_gen_var_94[t] + model.solar_gen_var_214[t] + model.solar_gen_var_38[t] \
               + model.solar_gen_var_21[t] + model.solar_gen_var_112[t] + model.solar_gen_var_107[t] \
               + model.solar_gen_var_107[t] + model.solar_gen_var_213[t] + model.wind_gen_var_1277[t] \
               + model.wind_gen_var_154[t] + model.wind_gen_var_1700[t] + model.wind_gen_var_1737[t] \
               + model.wind_gen_var_1804point5[t] + model.wind_gen_var_1804[t] + model.wind_gen_var_2048[t] \
               + model.wind_gen_var_209[t] + model.wind_gen_var_249[t] + model.wind_gen_var_263[t] \
               + model.wind_gen_var_356[t] + model.wind_gen_var_523[t] + model.wind_gen_var_530[t] \
               + model.wind_gen_var_622[t] + model.wind_gen_var_739[t] + model.ng_gen_var[t] for t in model.time_set_summer) \
               + sum(model.batt_soc_var[t] - model.batt_soc_var[t-1] for t in range(24,48)) \
                     >= sum(dict_demand[t] for t in model.time_set_summer)
model.demand_condition_day2_eql = pyo.Constraint(rule=demand_must_meet_day2)

#def demand_must_meet_day2 (self,t):
#    return sum(model.solar_gen_var[t] + model.wind_gen_var_1277[t] \
#               + model.wind_gen_var_154[t] + model.wind_gen_var_1700[t] + model.wind_gen_var_1737[t] \
#               + model.wind_gen_var_1804point5[t] + model.wind_gen_var_1804[t] + model.wind_gen_var_2048[t] \
#               + model.wind_gen_var_209[t] + model.wind_gen_var_249[t] + model.wind_gen_var_263[t] \
#               + model.wind_gen_var_356[t] + model.wind_gen_var_523[t] + model.wind_gen_var_530[t] \
#               + model.wind_gen_var_622[t] + model.wind_gen_var_739[t] + model.ng_gen_var[t] \
#               + model.batt_soc_var[t] - model.batt_soc_var[t-1] \
#               for t in range(1,24)) >= sum(dict_demand[t] for t in range(24,48))    
#    
#    return sum(model.solar_gen_var[t] + model.wind_gen_var[t] + model.ng_gen_var[t] + model.batt_soc_var[t] - model.batt_soc_var[t-1] for t in range(24,48)) \
#>= sum(dict_demand[t] for t in range(24,48))
#model.demand_condition_day2_eql = pyo.Constraint(rule=demand_must_meet_day2)

#model.demand_must_meet_day1_eql = pyo.Constraint(rule=demand_must_meet_day1)    


In [18]:
def power_balance_rule (self,t):
    return  dict_demand[t] + model.batt_charging_rate[t] <= (model.solar_gen_var_92[t] + model.solar_gen_var_177[t] \
                                                             + model.solar_gen_var_110[t] + model.solar_gen_var_94[t] + model.solar_gen_var_214[t] \
                                                             + model.solar_gen_var_38[t] + model.solar_gen_var_21[t] + model.solar_gen_var_112[t] \
                                                             + model.solar_gen_var_107[t] + model.solar_gen_var_213[t] \
                                                             + model.wind_gen_var_1277[t] + model.wind_gen_var_154[t] + model.wind_gen_var_1700[t] \
                                                             + model.wind_gen_var_1737[t] + model.wind_gen_var_1804point5[t] + model.wind_gen_var_1804[t] \
                                                             + model.wind_gen_var_2048[t] + model.wind_gen_var_209[t] + model.wind_gen_var_249[t] \
                                                             + model.wind_gen_var_263[t] + model.wind_gen_var_356[t] + model.wind_gen_var_523[t] + model.wind_gen_var_530[t] \
                                                             + model.wind_gen_var_622[t] + model.wind_gen_var_739[t] + model.ng_gen_var[t] + model.batt_discharging_rate[t])
model.power_balance_eql = pyo.Constraint(model.time_set, rule=power_balance_rule)


In [19]:
costs = sum(model.wind_gen_var_1277[t] * wind_capacity_var_1277_lcoe + model.wind_gen_var_154[t] * wind_capacity_var_154_lcoe + model.wind_gen_var_1700[t] * wind_capacity_var_1700_lcoe \
            + model.wind_gen_var_1737[t] * wind_capacity_var_1737_lcoe + model.wind_gen_var_1804point5[t] * wind_capacity_var_1804point5_lcoe + model.wind_gen_var_1804[t] * wind_capacity_var_1804_lcoe \
            + model.wind_gen_var_2048[t] * wind_capacity_var_2048_lcoe + model.wind_gen_var_209[t] * wind_capacity_var_209_lcoe + model.wind_gen_var_249[t] * wind_capacity_var_249_lcoe \
            + model.wind_gen_var_263[t] * wind_capacity_var_263_lcoe + model.wind_gen_var_356[t] * wind_capacity_var_356_lcoe + model.wind_gen_var_523[t] * wind_capacity_var_523_lcoe \
            + model.wind_gen_var_530[t] * wind_capacity_var_530_lcoe + model.wind_gen_var_622[t] * wind_capacity_var_622_lcoe + model.wind_gen_var_739[t] * wind_capacity_var_739_lcoe \
            + model.solar_gen_var_92[t]  * solar_capacity_var_92_lcoe \
            + model.solar_gen_var_177[t] * solar_capacity_var_170_lcoe   \
            + model.solar_gen_var_110[t] * solar_capacity_var_110_lcoe \
            + model.solar_gen_var_94[t]  * solar_capacity_var_94_lcoe  \
            + model.solar_gen_var_214[t] * solar_capacity_var_214_lcoe \
            + model.solar_gen_var_38[t]  * solar_capacity_var_38_lcoe  \
            + model.solar_gen_var_21[t]  * solar_capacity_var_21_lcoe  \
            + model.solar_gen_var_112[t] * solar_capacity_var_112_lcoe \
            + model.solar_gen_var_107[t] * solar_capacity_var_107_lcoe \
            + model.solar_gen_var_213[t] * solar_capacity_var_213_lcoe \
            + model.ng_gen_var[t] * ng_lcoe + \
            model.batt_discharging_rate[t] * battery_lcoe * discharging_eff for t in model.time_set)
model.objective_function = pyo.Objective(expr = costs, sense = minimize)


In [20]:
results = solver.solve(model, tee=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Bader\AppData\Local\Temp\tmp6z8jvp68.cplex.log' open.
CPLEX> Problem 'C:\Users\Bader\AppData\Local\Temp\tmpn556gya9.pyomo.lp' read.
Read time = 0.00 sec. (0.16 ticks)
CPLEX> Problem name         : C:\Users\Bader\AppData\Local\Temp\tmpn556gya9.pyomo.lp
Objective sense      : Minimize
Variables            :    1420
Objective nonzeros   :    1296
Linear constraints   :    1709  [Less: 361,  Greater: 2,  Equal: 1346]
  Nonzeros           :    5644
  RHS nonzeros       :     124

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : Min   : 0.04782

In [21]:
total_wind_capacity = (
    value(model.wind_capacity_var_1277      ) +
    value(model.wind_capacity_var_154       ) +
    value(model.wind_capacity_var_1700      ) +
    value(model.wind_capacity_var_1737      ) +
    value(model.wind_capacity_var_1804point5) +
    value(model.wind_capacity_var_1804      ) +
    value(model.wind_capacity_var_2048      ) +
    value(model.wind_capacity_var_209       ) +
    value(model.wind_capacity_var_249       ) +
    value(model.wind_capacity_var_263       ) +
    value(model.wind_capacity_var_356       ) +
    value(model.wind_capacity_var_523       ) +
    value(model.wind_capacity_var_530       ) +
    value(model.wind_capacity_var_622       ) +
    value(model.wind_capacity_var_739       ))/1000

total_solar_capacity = (
value(model.solar_capacity_var_92 ) + 
value(model.solar_capacity_var_177) +
value(model.solar_capacity_var_110) +
value(model.solar_capacity_var_94 ) + 
value(model.solar_capacity_var_214) +
value(model.solar_capacity_var_38 ) + 
value(model.solar_capacity_var_21 ) + 
value(model.solar_capacity_var_112) +
value(model.solar_capacity_var_107) +
value(model.solar_capacity_var_213))/1000

print("Solar Capacity: " + str(total_solar_capacity) +\
      "\nWind Capacity: " + str(total_wind_capacity) +\
      "\nBattery Capacity: " + str(value(model.batt_kWh_var)/1000) + \
      "\nNatural Gas Capacity factor: " + str(sum(value(model.ng_gen_var[t]) for t in model.time_set)/natgas_capacity_var))


print(
"wind_capacity_var_1277      : " + str(value(model.wind_capacity_var_1277       )/1000) + "\n"
"wind_capacity_var_154       : " + str(value(model.wind_capacity_var_154        )/1000) + "\n"
"wind_capacity_var_1700      : " + str(value(model.wind_capacity_var_1700       )/1000) + "\n"
"wind_capacity_var_1737      : " + str(value(model.wind_capacity_var_1737       )/1000) + "\n"
"wind_capacity_var_1804point5: " + str(value(model.wind_capacity_var_1804point5 )/1000) + "\n"
"wind_capacity_var_1804      : " + str(value(model.wind_capacity_var_1804       )/1000) + "\n"
"wind_capacity_var_2048      : " + str(value(model.wind_capacity_var_2048       )/1000) + "\n"
"wind_capacity_var_209       : " + str(value(model.wind_capacity_var_209        )/1000) + "\n"
"wind_capacity_var_249       : " + str(value(model.wind_capacity_var_249        )/1000) + "\n"
"wind_capacity_var_263       : " + str(value(model.wind_capacity_var_263        )/1000) + "\n"
"wind_capacity_var_356       : " + str(value(model.wind_capacity_var_356        )/1000) + "\n"
"wind_capacity_var_523       : " + str(value(model.wind_capacity_var_523        )/1000) + "\n"
"wind_capacity_var_530       : " + str(value(model.wind_capacity_var_530        )/1000) + "\n"
"wind_capacity_var_622       : " + str(value(model.wind_capacity_var_622        )/1000) + "\n"
"wind_capacity_var_739       : " + str(value(model.wind_capacity_var_739        )/1000) + "\n"
)

print(
"model.solar_capacity_var_92 : " + str(value(model.solar_capacity_var_92 ) / 1000) + "\n" 
"model.solar_capacity_var_177 : " + str(value(model.solar_capacity_var_177) / 1000) + "\n"
"model.solar_capacity_var_110 : " + str(value(model.solar_capacity_var_110) / 1000) + "\n"
"model.solar_capacity_var_94 : " + str(value(model.solar_capacity_var_94 ) / 1000) + "\n" 
"model.solar_capacity_var_214 : " + str(value(model.solar_capacity_var_214) / 1000) + "\n"
"model.solar_capacity_var_38 : " + str(value(model.solar_capacity_var_38 ) / 1000) + "\n" 
"model.solar_capacity_var_21 : " + str(value(model.solar_capacity_var_21 ) / 1000) + "\n" 
"model.solar_capacity_var_112 : " + str(value(model.solar_capacity_var_112) / 1000) + "\n"
"model.solar_capacity_var_107 : " + str(value(model.solar_capacity_var_107) / 1000) + "\n"
"model.solar_capacity_var_213 : " + str(value(model.solar_capacity_var_213) / 1000) + "\n"
)

Solar Capacity: 1178.0
Wind Capacity: 8494.5152015702
Battery Capacity: 3761.4411339326384
Natural Gas Capacity factor: 21.25962491617071
wind_capacity_var_1277      : 1277.0
wind_capacity_var_154       : 154.0
wind_capacity_var_1700      : 0.0
wind_capacity_var_1737      : 0.0
wind_capacity_var_1804point5: 1804.5
wind_capacity_var_1804      : 591.0152015701999
wind_capacity_var_2048      : 2048.0
wind_capacity_var_209       : 209.0
wind_capacity_var_249       : 0.0
wind_capacity_var_263       : 263.0
wind_capacity_var_356       : 356.0
wind_capacity_var_523       : 523.0
wind_capacity_var_530       : 530.0
wind_capacity_var_622       : 0.0
wind_capacity_var_739       : 739.0

model.solar_capacity_var_92 : 92.0
model.solar_capacity_var_177 : 177.0
model.solar_capacity_var_110 : 110.0
model.solar_capacity_var_94 : 94.0
model.solar_capacity_var_214 : 214.0
model.solar_capacity_var_38 : 38.0
model.solar_capacity_var_21 : 21.0
model.solar_capacity_var_112 : 112.0
model.solar_capacity_var_1

In [23]:
optimised = pd.DataFrame(columns = ['time', 'Total Solar Generation', 'Total Wind Generation', 'Natural Gas Generation', 'Discharging rate', 'Charging rate','Demand',  'Battery State of Charge', 'wind_gen_var_1277', 'wind_gen_var_154', 'wind_gen_var_1700' \
                                    ,'wind_gen_var_1737', 'wind_gen_var_1804point5', 'wind_gen_var_1804', 'wind_gen_var_2048' \
                                    , 'wind_gen_var_209', 'wind_gen_var_249', 'wind_gen_var_263', 'wind_gen_var_356' \
                                    , 'wind_gen_var_523', 'wind_gen_var_530', 'wind_gen_var_622', 'wind_gen_var_739' \
                                    , 'solar_capacity_var_92', 'solar_capacity_var_177','solar_capacity_var_110','solar_capacity_var_94', 'solar_capacity_var_214','solar_capacity_var_38','solar_capacity_var_21', 'solar_capacity_var_112','solar_capacity_var_107','solar_capacity_var_213'])
for t in model.time_set:
    if(t > 23):
        time = t - 24
    else:
        time = t
    entry = {'time': time,\
    'Demand' : dict_demand[t]/1000,\
    'wind_gen_var_1277'       : value(model.wind_gen_var_1277[t]       / 1000),\
    'wind_gen_var_154'        : value(model.wind_gen_var_154[t]        / 1000),\
    'wind_gen_var_1700'       : value(model.wind_gen_var_1700[t]       / 1000),\
    'wind_gen_var_1737'       : value(model.wind_gen_var_1737[t]       / 1000),\
    'wind_gen_var_1804point5' : value(model.wind_gen_var_1804point5[t] / 1000),\
    'wind_gen_var_1804'       : value(model.wind_gen_var_1804[t]       / 1000),\
    'wind_gen_var_2048'       : value(model.wind_gen_var_2048[t]       / 1000),\
    'wind_gen_var_209'        : value(model.wind_gen_var_209[t]        / 1000),\
    'wind_gen_var_249'        : value(model.wind_gen_var_249[t]        / 1000),\
    'wind_gen_var_263'        : value(model.wind_gen_var_263[t]        / 1000),\
    'wind_gen_var_356'        : value(model.wind_gen_var_356[t]        / 1000),\
    'wind_gen_var_523'        : value(model.wind_gen_var_523[t]        / 1000),\
    'wind_gen_var_530'        : value(model.wind_gen_var_530[t]        / 1000),\
    'wind_gen_var_622'        : value(model.wind_gen_var_622[t]        / 1000),\
    'wind_gen_var_739'        : value(model.wind_gen_var_739[t]        / 1000),\
    'solar_gen_var_92'        : value(model.solar_gen_var_92 [t]       / 1000), \
    'solar_gen_var_177'       : value(model.solar_gen_var_177[t]       / 1000), \
    'solar_gen_var_110'       : value(model.solar_gen_var_110[t]       / 1000), \
    'solar_gen_var_94'        : value(model.solar_gen_var_94 [t]       / 1000), \
    'solar_gen_var_214'       : value(model.solar_gen_var_214[t]       / 1000), \
    'solar_gen_var_38'        : value(model.solar_gen_var_38 [t]       / 1000), \
    'solar_gen_var_21'        : value(model.solar_gen_var_21 [t]       / 1000), \
    'solar_gen_var_112'       : value(model.solar_gen_var_112[t]       / 1000), \
    'solar_gen_var_107'       : value(model.solar_gen_var_107[t]       / 1000), \
    'solar_gen_var_213'       : value(model.solar_gen_var_213[t]       / 1000), \
    'Total Wind Generation'   : (value(model.wind_gen_var_1277[t]) + value(model.wind_gen_var_154[t]) +value(model.wind_gen_var_1700[t]) + value(model.wind_gen_var_1737[t]) + value(model.wind_gen_var_1804point5[t]) + value(model.wind_gen_var_1804[t]) + value(model.wind_gen_var_2048[t]) + value(model.wind_gen_var_209[t]) + value(model.wind_gen_var_249[t]) + value(model.wind_gen_var_263[t]) + value(model.wind_gen_var_356[t]) + value(model.wind_gen_var_523[t]) + value(model.wind_gen_var_530[t]) + value(model.wind_gen_var_622[t]) + value(model.wind_gen_var_739[t])) / 1000,\
    'Total Solar Generation'  : (value(model.solar_gen_var_92[t]) + value(model.solar_gen_var_177[t]) + value(model.solar_gen_var_110[t]) + value(model.solar_gen_var_94 [t]) + value(model.solar_gen_var_214[t]) + value(model.solar_gen_var_38 [t]) + value(model.solar_gen_var_21 [t]) + value(model.solar_gen_var_112[t]) + value(model.solar_gen_var_107[t]) + value(model.solar_gen_var_213[t]))/1000,\
    'Charging rate'           : value(model.batt_charging_rate[t])/1000,\
    'Discharging rate'        : value(model.batt_discharging_rate[t])/1000,\
    'Battery State of Charge' : value(model.batt_soc_var[t])/1000,\
    'Natural Gas Generation'  : value(model.ng_gen_var[t])/1000 
    }
#    entry = {'time': time, 'Demand' : dict_demand[t]/1000000, 'Solar Generation' : value(model.solar_gen_var[t])/1000000\
#             , 'Wind Generation' : value(model.wind_gen_var[t])/1000000, \
#             'Charging rate' : value(model.batt_charging_rate[t])/1000000, 'Discharging rate' : value(model.batt_discharging_rate[t])/1000000, \
#             'Battery State of Charge' : value(model.batt_soc_var[t])/1000000, 'Natural Gas Generation' : value(model.ng_gen_var[t])/1000000}
    optimised = optimised.append(entry, ignore_index = True)

In [24]:
optimised['Non Natural Gas Demand'] = (optimised['Demand'] - optimised['Natural Gas Generation']) / optimised['Demand']

In [26]:
optimised

,time,Total Solar Generation,Total Wind Generation,Natural Gas Generation,Discharging rate,Charging rate,Demand,Battery State of Charge,wind_gen_var_1277,wind_gen_var_154,...,solar_gen_var_110,solar_gen_var_112,solar_gen_var_177,solar_gen_var_21,solar_gen_var_213,solar_gen_var_214,solar_gen_var_38,solar_gen_var_92,solar_gen_var_94,Non Natural Gas Demand
0,0.0,0.000000,4267.182704,0.000000,0.000000,0.000000,2350.944586,0.000000,638.329733,76.543133,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,1.0,0.000000,4157.187004,0.000000,0.000000,0.000000,2260.337694,0.000000,618.706500,74.310133,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,2.0,0.000000,4056.221546,0.000000,0.000000,0.000000,2176.034865,0.000000,600.956200,72.451867,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,3.0,0.000000,3947.423859,0.000000,0.000000,0.000000,2043.066236,0.000000,583.589000,70.204129,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,4.0,0.000000,3892.623184,0.000000,0.000000,0.000000,2026.117054,0.000000,577.451161,69.066516,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,5.0,0.000000,3875.766440,0.000000,0.000000,0.000000,2076.156961,0.000000,584.618839,68.520065,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
6,6.0,47.769710,3794.295295,0.000000,0.000000,0.000000,2113.953300,0.000000,576.050581,67.059548,...,4.435484,4.545032,7.211322,0.861000,8.609323,8.677355,1.534710,3.703742,3.835806,1.000000
7,7.0,385.626193,3503.558765,0.000000,0.000000,0.000000,2078.568658,0.000000,552.652645,62.171290,...,35.941613,36.522839,58.187322,6.898839,69.767806,70.219613,12.411290,29.953419,30.713742,1.000000
8,8.0,625.645194,3264.540482,0.000000,0.000000,0.000000,2220.337177,0.000000,531.232000,58.251742,...,58.324839,59.284129,94.369548,11.182161,113.206065,113.923936,20.141226,48.676903,49.798774,1.000000
9,9.0,759.343839,3273.535352,0.000000,0.000000,0.000000,2362.077652,0.000000,520.398097,58.832968,...,70.950000,72.073806,114.204968,13.535516,137.364387,138.057613,24.487935,59.215355,60.511742,1.000000


In [28]:
#optimised.to_csv('Low Case With All Sites.csv')